***Regression***

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [4]:
import os
path = "gdrive/My Drive/Colab Notebooks/01_Test/data/"
os.listdir(path)

['titanic.csv', 'titanic_mini.csv']

In [5]:
import pandas as pd 
import numpy as np 

In [6]:
df = pd.read_csv(path + "titanic_mini.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,93,0,1,"Chaffee, Mr. Herbert Fuller",male,46.0,1,0,W.E.P. 5734,61.1750,E31,S
1,773,0,2,"Mack, Mrs. (Mary)",female,57.0,0,0,S.O./P.P. 3,10.5000,E77,S
2,488,0,1,"Kent, Mr. Edward Austin",male,58.0,0,0,11771,29.7000,B37,C
3,300,1,1,"Baxter, Mrs. James (Helene DeLaudeniere Chaput)",female,50.0,0,1,PC 17558,247.5208,B58 B60,C
4,660,0,1,"Newell, Mr. Arthur Webster",male,58.0,0,2,35273,113.2750,D48,C
5,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
6,22,1,2,"Beesley, Mr. Lawrence",male,34.0,0,0,248698,13.0000,D56,S
7,453,0,1,"Foreman, Mr. Benjamin Laventall",male,30.0,0,0,113051,27.7500,C111,C
8,663,0,1,"Colley, Mr. Edward Pomeroy",male,47.0,0,0,5727,25.5875,E58,S
9,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S


In [ ]:
len(df)
df.columns


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
X_column = ['Survived', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']
Y_Column = 'Age'

In [ ]:
df['Survived'].unique()

array([0, 1])

Survieved=0 고령층 -> Age = 나이 높게 예측 
Survieved=1 젋은 사람 -> Age = 나이 젊게 예측 

데이터 전체 개수 = Unique한 종류 개수 ---> 모델 학습에 도움이 되지 않음, 삭제 

In [ ]:
print(df['Survived'].unique())
print(df['Survived'].nunique())
# 사용가능한 데이터를 select 하는 과정 
# 1종류만 두 사람이 공유 -> 너무 치우친 값 (Biased) -> 정규분포, 다항분포일수록 학습이 잘됨, bias이거나 데이터가 전체개수와 같으면 학습에 도움이 안됨 
# 단한 분포 -> Uniform Distribution 
# 최종적으로 모델에 사용할 컬럼 (column, feature) 결정

[0 1]
2


In [ ]:
# 최종적으로 모델에 사용할 컬럼 (column, feature) 결정
input_data = df[X_column + [Y_Column]].copy() # Memory
input_data

,Survived,Pclass,Sex,SibSp,Parch,Embarked,Age
0,0,1,male,1,0,S,46.0
1,0,2,female,0,0,S,57.0
2,0,1,male,0,0,C,58.0
3,1,1,female,0,1,C,50.0
4,0,1,male,0,2,C,58.0
5,1,1,female,1,0,C,38.0
6,1,2,male,0,0,S,34.0
7,0,1,male,0,0,C,30.0
8,0,1,male,0,0,S,47.0
9,0,1,male,0,0,S,54.0


In [ ]:
# 모델에 입력 데이터 타입 변환 (numeric value 숫자만 가능하다, int, float)
input_data.dtypes 

Survived      int64
Pclass        int64
Sex          object
SibSp         int64
Parch         int64
Embarked     object
Age         float64
dtype: object

In [ ]:
input_data['Sex'] = input_data['Sex'].replace({'female' : 0, 'male' : 1})
input_data['Sex']

0     1
1     0
2     1
3     0
4     1
5     0
6     1
7     1
8     1
9     1
10    0
11    1
12    1
13    1
14    1
15    1
16    0
17    0
18    1
19    1
Name: Sex, dtype: int64

In [ ]:
input_data['Embarked'] = input_data['Embarked'].replace({'S' : 0, 'C' : 1})
input_data['Embarked']

0     0
1     0
2     1
3     1
4     1
5     1
6     0
7     1
8     0
9     0
10    1
11    0
12    1
13    0
14    1
15    0
16    0
17    0
18    0
19    1
Name: Embarked, dtype: int64

In [ ]:
input_data.dtypes

Survived      int64
Pclass        int64
Sex           int64
SibSp         int64
Parch         int64
Embarked      int64
Age         float64
dtype: object

In [ ]:
# 모델 입력을 할 수 있는 데이터 타입으로 변경되었음 
# 모델 학습을 위해 필요한 세 가지 데이터 
# 1. train data 학습 
#   -train data 
#   -validation data 학습 하는 도중에 실행하는 성능평가 
# 2. test data 모델 성능 평가 

# Our Total Data = 20 개 
# 14개 -> Train -> train(8), validation(6) 
# 6개 -> test 

# 데이터 -> train/valid , test  크게 두 가지로 나눔 
from sklearn.model_selection import train_test_split

tr_val_X, test_X, tr_val_Y, test_Y = train_test_split(
    input_data[X_column], # X 입력 데이터
    input_data[Y_Column], # Y 정답 데이터 
    test_size = 0.2
    #train_size = 13, 
    shuffle=True,
    random_state=42)


In [ ]:
tr_val_data = pd.concat([tr_val_X, tr_val_Y], axis=1)
tr_val_data

,Survived,Pclass,Sex,SibSp,Parch,Embarked,Age
8,0,1,1,0,0,0,47.0
5,1,1,0,1,0,1,38.0
11,0,3,1,0,0,0,19.0
3,1,1,0,0,1,1,50.0
18,0,1,1,1,0,0,50.0
16,1,1,0,0,1,0,58.0
13,0,1,1,0,0,0,39.0
2,0,1,1,0,0,1,58.0
9,0,1,1,0,0,0,54.0
19,1,1,1,0,1,1,23.0


In [ ]:
from sklearn.model_selection import train_test_split

train_X, valid_X, train_Y, valid_Y = train_test_split(
    tr_val_data[X_column], # X 입력 데이터
    tr_val_data[Y_Column], # Y 정답 데이터 
    test_size = 0.2, 
    shuffle=True,
    random_state=42)

In [ ]:
print(len(train_X))
print(len(valid_X))
print(len(test_X))
len(train_X) + len(valid_X) + len(test_X) # 20


12
4
4


20

In [ ]:
# 본격적인 모델 학습의 시작! 

In [ ]:
# Decision Tree, Random Forest, XGB, CATBOOST 
# 이 중 하나를 가지고 졸업작품을 하면 됨 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model_rf = RandomForestRegressor()
model_rf.fit(train_X, train_Y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
display(test_X, test_Y)

,Survived,Pclass,Sex,SibSp,Parch,Embarked
0,0,1,1,1,0,0
17,1,2,0,0,0,0
15,0,2,1,0,2,0
1,0,2,0,0,0,0


0     46.0
17    29.0
15    36.5
1     57.0
Name: Age, dtype: float64

In [ ]:
true_y = test_Y
true_y
# 의미를 이해하기 쉽게 변수 명 설정 

0     46.0
17    29.0
15    36.5
1     57.0
Name: Age, dtype: float64

In [ ]:
# prediction (예측)
pred_y = model_rf.predict(test_X)
pred_y

array([48.43      , 37.265     , 48.1225    , 40.57133333])

In [ ]:
df_result = pd.DataFrame(list(zip(test_Y, pred_y)), columns=['true_y', 'pred_y'])
df_result


,true_y,pred_y
0,46.0,48.430000
1,29.0,37.265000
2,36.5,48.122500
3,57.0,40.571333


In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(true_y, pred_y)
mse

119.79967992361111

In [ ]:
# RMSE root mean, 에러율
rmse = np.sqrt(mse)
rmse

10.945304012388652

In [ ]:
from xgboost import XGBRegressor

model_xgb = XGBRegressor()
model_xgb.fit(train_X, train_Y, eval_set=[(valid_X, valid_Y)])


[03:11:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:44.3522
[1]	validation_0-rmse:41.0197
[2]	validation_0-rmse:37.9999
[3]	validation_0-rmse:35.2644
[4]	validation_0-rmse:32.8644
[5]	validation_0-rmse:30.6151
[6]	validation_0-rmse:28.5801
[7]	validation_0-rmse:26.7967
[8]	validation_0-rmse:25.2461
[9]	validation_0-rmse:23.7229
[10]	validation_0-rmse:22.4325
[11]	validation_0-rmse:21.2149
[12]	validation_0-rmse:20.1422
[13]	validation_0-rmse:19.1541
[14]	validation_0-rmse:18.2235
[15]	validation_0-rmse:17.3767
[16]	validation_0-rmse:16.6017
[17]	validation_0-rmse:15.8987
[18]	validation_0-rmse:15.2777
[19]	validation_0-rmse:14.7117
[20]	validation_0-rmse:14.1964
[21]	validation_0-rmse:13.7276
[22]	validation_0-rmse:13.3016
[23]	validation_0-rmse:12.9147
[24]	validation_0-rmse:12.5636
[25]	validation_0-rmse:12.2452
[26]	validation_0-rmse:11.9753
[27]	validation_0-rmse:11.7122
[28]	validation

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
# prediction (예측)
xgb_pred_y = model_xgb.predict(test_X)